In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, GroupKFold, train_test_split
import re
from sklearn.preprocessing import LabelEncoder
#from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from lightgbm import LGBMRegressor
from sklearn.cluster import KMeans
import lightgbm as lgb
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.decomposition import PCA
plt.rcParams['figure.figsize'] = [5, 5]
pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
train.head()

## target variable class

In [ ]:
train['target'].value_counts()

In [ ]:
### shape of the data
train.shape, test.shape

In [ ]:
## storing the id
test_id = test['id']

test = test.drop('id', axis=1)
train = train.drop('id', axis=1)

In [ ]:
## checking the categorical features
train.dtypes[train.dtypes == 'object'].index

To make our baseline model easier we are going to use Lightgbm categorical data encoding

In [ ]:
categorical_feats = [
    f for f in train.columns if train[f].dtype == 'object'
]

categorical_feats
for f_ in categorical_feats:
    train[f_] = train[f_].astype('category')

In [ ]:
categorical_feats = [
    f for f in test.columns if test[f].dtype == 'object'
]

categorical_feats
for f_ in categorical_feats:
    test[f_] = test[f_].astype('category')

In [ ]:
X = train.drop('target', axis=1)
y = train['target']

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 5, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 5,
         "min_sum_hessian_in_leaf":10,
         "boosting": "gbdt",
         "feature_fraction": 0.9,'early_stopping':100,
         "bagging_freq": 1,
         "bagging_fraction": 0.85 ,
         "bagging_seed": 224,
         "metric": 'auc',
         "verbosity": -1}

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
fold= StratifiedKFold(n_splits=5, shuffle=True, random_state=33)#10-1
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m = LGBMClassifier(**param,n_estimators=2000)
    m.fit(X_train,y_train,categorical_feature=categorical_feats,eval_set=[(X_train,y_train),(X_test, y_test)],verbose=-1, early_stopping_rounds=100)#,verbose=100)
    preds=m.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:, 1]
    y_pred_totcb.append(p)

In [ ]:
np.mean(errcb)

In [ ]:
fea_imp = pd.DataFrame({'imp':m.feature_importances_, 'col': X.columns})
fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=False).iloc[-30:]
_ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))

In [ ]:
d = {"id": test_id, 'target': (np.mean(y_pred_totcb, 0))}
sub = pd.DataFrame(data=d)
sub = sub[["id", 'target']].round(4) # to have a slight better score

In [ ]:
sub.head()

In [ ]:
sub.to_csv('marb.csv', index=False)